In [27]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate 
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature = 0.1
    , streaming = True
    , callbacks = [StreamingStdOutCallbackHandler()]
)

In [28]:
# 제공하는 예제 -> model이 이런 식으로 답변하면 좋겠다.
from typing import Any, Dict


examples = [
    {
    "question": "France?",
    "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
    """,
    },
    {
    "question": "Italy?",
    "answer": '''
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
    ''',
    },
    {
    "question": "Greece?",
    "answer": '''
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
    ''',
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [30]:
example_prompt = PromptTemplate.from_template(
"""
    Human:{question}
    AI:{answer}"
"""
)

example_selector = RandomExampleSelector(
    examples       = examples
)

prompt = FewShotPromptTemplate(
    example_prompt   = example_prompt,
    example_selector = example_selector,
    suffix           = "Human: What do you know about {country}?",
    input_variables  = ["country"]
)

prompt.format(country="Japan")

# chain = prompt | chat
# chain.invoke({
#     "country":"Taiwan"
# })

'\n    Human:Italy?\n    AI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n    "\n\n\nHuman: What do you know about Japan?'